# Store Dataset

In [1]:
import cv2
import os
from glob import glob
from tqdm.auto import tqdm
import shutil
import matplotlib.pyplot as plt
import numpy as np
import math
import os.path as osp
import pyclipper
from shapely.geometry import Polygon
#mean = [103.939, 116.779, 123.68]
# thresh_min=0.3
# thresh_max=0.7
#-----------------------------------
# thresh_map = thresh_map * (thresh_max - thresh_min) + thresh_min
# image = img.astype(np.float32)
# image[..., 0] -= mean[0]
# image[..., 1] -= mean[1]
# image[..., 2] -= mean[2]
#-----------------------------------
min_text_size=8
shrink_ratio=0.4
IMAGE_SIZE=512
data_dir=os.path.join(os.getcwd(),"data")
img_dir=os.path.join(data_dir,"image")
gt_dir=os.path.join(data_dir,"gt")
mask_dir=os.path.join(data_dir,"mask")
thresh_mask_dir=os.path.join(data_dir,"thresh_mask")
thresh_map_dir=os.path.join(data_dir,"thresh_map")

ds_idens=os.listdir("datasets")
fiden=0

In [2]:


def draw_thresh_map(polygon, canvas, mask, shrink_ratio=0.4):
    assert polygon.ndim == 2
    assert polygon.shape[1] == 2

    polygon_shape = Polygon(polygon)
    distance = polygon_shape.area * (1 - np.power(shrink_ratio, 2)) / polygon_shape.length
    subject = [tuple(l) for l in polygon]
    padding = pyclipper.PyclipperOffset()
    padding.AddPath(subject, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    padded_polygon = np.array(padding.Execute(distance)[0])
    cv2.fillPoly(mask, [padded_polygon.astype(np.int32)], 1.0)

    xmin = padded_polygon[:, 0].min()
    xmax = padded_polygon[:, 0].max()
    ymin = padded_polygon[:, 1].min()
    ymax = padded_polygon[:, 1].max()
    width = xmax - xmin + 1
    height = ymax - ymin + 1

    polygon[:, 0] = polygon[:, 0] - xmin
    polygon[:, 1] = polygon[:, 1] - ymin

    xs = np.broadcast_to(np.linspace(0, width - 1, num=width).reshape(1, width), (height, width))
    ys = np.broadcast_to(np.linspace(0, height - 1, num=height).reshape(height, 1), (height, width))

    distance_map = np.zeros((polygon.shape[0], height, width), dtype=np.float32)
    for i in range(polygon.shape[0]):
        j = (i + 1) % polygon.shape[0]
        absolute_distance = compute_distance(xs, ys, polygon[i], polygon[j])
        distance_map[i] = np.clip(absolute_distance / distance, 0, 1)
    distance_map = np.min(distance_map, axis=0)

    xmin_valid = min(max(0, xmin), canvas.shape[1] - 1)
    xmax_valid = min(max(0, xmax), canvas.shape[1] - 1)
    ymin_valid = min(max(0, ymin), canvas.shape[0] - 1)
    ymax_valid = min(max(0, ymax), canvas.shape[0] - 1)
    canvas[ymin_valid:ymax_valid + 1, xmin_valid:xmax_valid + 1] = np.fmax(
        1 - distance_map[
            ymin_valid - ymin:ymax_valid - ymin+1,
            xmin_valid - xmin:xmax_valid - xmin+1],
        canvas[ymin_valid:ymax_valid + 1, xmin_valid:xmax_valid + 1])
    
def compute_distance(xs, ys, point_1, point_2):
    square_distance_1 = np.square(xs - point_1[0]) + np.square(ys - point_1[1])
    square_distance_2 = np.square(xs - point_2[0]) + np.square(ys - point_2[1])
    square_distance = np.square(point_1[0] - point_2[0]) + np.square(point_1[1] - point_2[1])

    cosin = (square_distance - square_distance_1 - square_distance_2) / \
            (2 * np.sqrt(square_distance_1 * square_distance_2))
    square_sin = 1 - np.square(cosin)
    square_sin = np.nan_to_num(square_sin)
    result = np.sqrt(square_distance_1 * square_distance_2 * square_sin / square_distance)

    result[cosin < 0] = np.sqrt(np.fmin(square_distance_1, square_distance_2))[cosin < 0]
    return result


def get_anns(gt_path,ds_iden):
    lines = []
    # ann
    reader = open(gt_path, 'r').readlines()
    for line in reader:
        item = {}
        parts = line.strip().split(',')
        label = parts[-1]
        # text
        if label == '1':
            label = '###'
        line = [i.strip('\ufeff').strip('\xef\xbb\xbf') for i in parts]

        if 'icdar' in ds_iden:
            poly = np.array(list(map(float, line[:8]))).reshape((-1, 2)).tolist()
        else:
            num_points = math.floor((len(line) - 1) / 2) * 2
            poly = np.array(list(map(float, line[:num_points]))).reshape((-1, 2)).tolist()
        if len(poly) < 3:
            continue
        item['poly'] = poly
        item['text'] = label
        lines.append(item)
    return lines

def resize(size, image, anns):
    h, w, c = image.shape
    scale_w = size / w
    scale_h = size / h
    scale = min(scale_w, scale_h)
    h = int(h * scale)
    w = int(w * scale)
    padimg = np.zeros((size, size, c), image.dtype)
    padimg[:h, :w] = cv2.resize(image, (w, h))
    new_anns = []
    for ann in anns:
        poly = np.array(ann['poly']).astype(np.float64)
        poly *= scale
        new_ann = {'poly': poly.tolist(), 'text': ann['text']}
        new_anns.append(new_ann)
    return padimg, new_anns

def process_single_data(img_path,ds_iden):
    # img
    image=cv2.imread(img_path)
    # masks
    dim=(IMAGE_SIZE,IMAGE_SIZE)
    gt = np.zeros(dim, dtype=np.float32)
    mask = np.ones(dim, dtype=np.float32)
    thresh_map = np.zeros(dim, dtype=np.float32)
    thresh_mask = np.zeros(dim, dtype=np.float32)
    # gt_path
    if ds_iden!="icdar2":
        gt_path=img_path.replace("images","gts")+".txt"
    else:
        gt_path=img_path.replace("images","gts").replace("img","gt_img")
        gt_path=gt_path.split(".")[0]+".txt"
    anns=get_anns(gt_path,ds_iden)
    # resize
    image, anns = resize(IMAGE_SIZE, image, anns)
    # process annotations
    for ann in anns:
        poly = np.array(ann['poly'])
        height = max(poly[:, 1]) - min(poly[:, 1])
        width = max(poly[:, 0]) - min(poly[:, 0])
        polygon = Polygon(poly)
        # generate gt and mask
        if polygon.area < 1 or min(height, width) < min_text_size or ann['text'] == '###':
            cv2.fillPoly(mask, poly.astype(np.int32)[np.newaxis, :, :], 0)
            continue
        else:
            distance = polygon.area * (1 - np.power(shrink_ratio, 2)) / polygon.length
            subject = [tuple(l) for l in ann['poly']]
            padding = pyclipper.PyclipperOffset()
            padding.AddPath(subject, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
            shrinked = padding.Execute(-distance)
            if len(shrinked) == 0:
                cv2.fillPoly(mask, poly.astype(np.int32)[np.newaxis, :, :], 0)
                continue
            else:
                shrinked = np.array(shrinked[0]).reshape(-1, 2)
                if shrinked.shape[0] > 2 and Polygon(shrinked).is_valid:
                    cv2.fillPoly(gt, [shrinked.astype(np.int32)], 1)
                else:
                    cv2.fillPoly(mask, poly.astype(np.int32)[np.newaxis, :, :], 0)
                    continue
        # generate thresh map and thresh mask
        draw_thresh_map(poly, thresh_map, thresh_mask, shrink_ratio=shrink_ratio)
    thresh_map*=255
    #thresh_map=thresh_map.astype("uint8")
    gt*=255
    mask*=255
    thresh_mask*=255
    return image,gt,mask,thresh_map,thresh_mask

In [3]:


def process_dataset(ds_path,ds_iden):
    global fiden
    dataset_path=os.path.join(ds_path,ds_iden)
    img_paths=[img_path for img_path in tqdm(glob(os.path.join(dataset_path,"images","*.*")))]
    # extract anns 
    for img_path in tqdm(img_paths):
        try:
            img,gt,mask,thresh_map,thresh_mask=process_single_data(img_path,ds_iden)
            # save
            cv2.imwrite(os.path.join(img_dir,f"{fiden}.png"),img)
            cv2.imwrite(os.path.join(gt_dir,f"{fiden}.png"),gt)
            cv2.imwrite(os.path.join(mask_dir,f"{fiden}.png"),mask)
            cv2.imwrite(os.path.join(thresh_map_dir,f"{fiden}.png"),thresh_map)
            cv2.imwrite(os.path.join(thresh_mask_dir,f"{fiden}.png"),thresh_mask)
            fiden+=1
        except Exception as e:
            print(e)

In [4]:
ds_idens

['td1', 'icdar1', 'totaltext', 'td2', 'icdar2', 'td3']

In [ ]:
for ds_iden in ds_idens:
    print(ds_iden)
    process_dataset("datasets/",ds_iden)

td1


  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

<ipython-input-2-372fac683a57>:52: RuntimeWarning: invalid value encountered in sqrt
  result = np.sqrt(square_distance_1 * square_distance_2 * square_sin / square_distance)
<ipython-input-2-372fac683a57>:48: RuntimeWarning: invalid value encountered in true_divide
  cosin = (square_distance - square_distance_1 - square_distance_2) / \


icdar1


...

  0%|          | 0/1555 [00:00<?, ?it/s]

<ipython-input-2-372fac683a57>:52: RuntimeWarning: invalid value encountered in true_divide
  result = np.sqrt(square_distance_1 * square_distance_2 * square_sin / square_distance)


td2


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
# ds_iden=ds_idens[0]
# dataset_path=os.path.join("datasets/",ds_iden)
# img_paths=[img_path for img_path in tqdm(glob(os.path.join(dataset_path,"images","*.*")))]
# img_path=img_paths[0]
# img_path
# plt.imshow(img)
# plt.show()
# plt.imshow(gt)
# plt.show()
# plt.imshow(mask)
# plt.show()
# plt.imshow(thresh_map)
# plt.show()
# plt.imshow(thresh_mask)
# plt.show()
# np.unique(thresh_map)